<a href="https://colab.research.google.com/github/HSS107048212/Burnout-Analysis-and-Prediction/blob/main/112034602_%E9%99%B3%E7%9A%86%E9%8C%AB_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

程式建立與迭代思維：

* 使用pandas庫讀取CSV文件，並檢查哪些列包含缺失值。
* 將日期轉換為距今的天數，以獲得新的特徵。
* 使用LabelEncoder對類別變數進行編碼，使其能夠在機器學習模型中使用。
* 使用SimpleImputer來填充缺失值，可以選擇不同的策略，如均值、中位數等。
  * 嘗試使用過線性回歸、平均數、中位數進行NA補充。
  * 其中「線性回歸」是採取最簡單單一變數，即Y=ax+b。而x是透過correlation找出來最相關的parameter帶入線性回歸補na。
  * 最後，經過測試median是比較好的補na方式。
* 將數據分為訓練集和驗證集，並對特徵進行標準化。
* 創建了一個深度學習模型，使用tensorflow和keras庫。
  * 這是三個稠密層（全連接層），包含128,64,1個神經元。前2層使用ReLU（Rectified Linear Unit）激活函數作為激活函數；最後一個稠密層使用線性激活函數。這通常用於回歸任務，其中模型的輸出是一個數值。
  * 有測試更多稠密層，包含1024,512,256個神經元，但效果沒有更好，所以最後只取用3層，其中最多128神經元。
* 並擔配使用「學習率衰減與提前停止」技巧，訓練深度學習模型。
* 讀取測試數據，處理缺失值並進行預測。
* 最後，將預測結果保存為CSV文件。到Kaggle測試分數。
* Fine-tune的流程是調整「learning_rate、dropout_rate、test_size、decay_steps、decay_rate」變數並觀察記錄public score，思考調參策略，進行迭代。

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression

# 讀取CSV檔案
data = pd.read_csv("train.csv")

# 检查哪些列存在缺失值
columns_with_missing = data.columns[data.isnull().any()]

# 显示具有缺失值的列
print("Columns with missing values in data:")
print(columns_with_missing)


# 處理缺失值: 使用中位數填充
data.fillna(data.median(), inplace=True)

# 將入職日期轉換為距今的天數
data['Days Since Joining'] = (pd.Timestamp.now() - pd.to_datetime(data['Date of Joining'])).dt.days
data.drop('Date of Joining', axis=1, inplace=True)

# 使用LabelEncoder對類別變數進行編碼
label_encoders = {}
categorical_columns = ['Gender', 'Company Type', 'WFH Setup Available']

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# 切分數據為訓練集和驗證集
X = data.drop(['Employee ID', 'Burn Rate'], axis=1)
y = data['Burn Rate']



def build_and_train_model(X_train, y_train, X_valid, y_valid, num_epochs=100, batch_size=32, learning_rate=0.01, dropout_rate=0.3):
    # 使用較複雜的深度學習模型，這裡使用的是建立模型的代碼
    model = keras.Sequential([
        layers.InputLayer(input_shape=(X_train.shape[1],)),
        #layers.Dense(1024, activation='relu'),
        #layers.Dropout(dropout_rate),
        #layers.Dense(512, activation='relu'),
        #layers.Dropout(dropout_rate),
        #layers.Dense(256, activation='relu'),
        #layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation='linear')
    ])

    # 使用學習率衰減
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=9500,
        decay_rate=0.7
    )
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

    # 編譯模型
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    # 使用提前停止和模型檢查點
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)

    # 訓練模型
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_valid, y_valid), callbacks=[early_stopping, model_checkpoint], verbose=1)

    return model, history


learning_rate =0.01
dropout_rate =0.1
test_size =0.32


# 切分數據為訓練集和驗證集
X = data.drop(['Employee ID', 'Burn Rate'], axis=1)
y = data['Burn Rate']
# 示例用法
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
model, history = build_and_train_model(X_train, y_train, X_valid, y_valid, num_epochs=100, batch_size=32, learning_rate=learning_rate, dropout_rate=dropout_rate)



# 讀取test.csv
test_data = pd.read_csv("test.csv")

# 检查哪些列存在缺失值
columns_with_missing = test_data.columns[test_data.isnull().any()]

# 显示具有缺失值的列
print("Columns with missing values in test_data:")
print(columns_with_missing)

# 處理缺失值: 使用中位數填充
test_data.fillna(data.median(), inplace=True)

# 將入職日期轉換為距今的天數
test_data['Days Since Joining'] = (pd.Timestamp.now() - pd.to_datetime(test_data['Date of Joining'])).dt.days
test_data.drop('Date of Joining', axis=1, inplace=True)

# 使用之前的LabelEncoder對類別變數進行編碼
for col in categorical_columns:
    test_data[col] = label_encoders[col].transform(test_data[col])

# 移除Employee ID並標準化特徵
X_test = test_data.drop('Employee ID', axis=1)
X_test = scaler.transform(X_test)

# 載入最佳模型權重
model.load_weights('best_model.h5')

# 進行預測
predictions = model.predict(X_test)

minimum_values = predictions.min()
print("Minimum values for all columns:")
print(minimum_values) #如果預測的離職率出現負數，則表示模型有問題。
# 将小于1的数值转换为0
predictions[predictions < 0] = 0

# 創建預測結果的DataFrame並保存為CSV
df_ans = pd.DataFrame(predictions, columns=['Burn Rate'])
df_ans.to_csv(f'mySubmission_learning_rate{learning_rate}_dropout_rate{dropout_rate}.csv', index_label='Employee ID')

